<a href="https://colab.research.google.com/github/okwasna/Bioinformatic-Needleman-Wunsch/blob/main/nw_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install biopython
from Bio import SeqIO


In [17]:
def needleman_wunsch(seq1, seq2, match_score, mismatch_penalty, gap_penalty):
    m, n = len(seq1), len(seq2)
    # Inicjalizacja macierzy skorowania
    score_matrix = [[0] * (n+1) for _ in range(m+1)]

    # Inicjalizacja macierzy wskazań kierunków (traceback matrix)
    traceback_matrix = [[None] * (n+1) for _ in range(m+1)]

    # Wypełnienie pierwszego wiersza i kolumny
    for i in range(m+1):
        score_matrix[i][0] = gap_penalty * i
        traceback_matrix[i][0] = 'up'
    for j in range(n+1):
        score_matrix[0][j] = gap_penalty * j
        traceback_matrix[0][j] = 'left'

    # Wypełnianie macierzy skorowania i traceback
    for i in range(1, m+1):
        for j in range(1, n+1):
            match = score_matrix[i-1][j-1] + (match_score if seq1[i-1] == seq2[j-1] else mismatch_penalty)
            delete = score_matrix[i-1][j] + gap_penalty
            insert = score_matrix[i][j-1] + gap_penalty
            score_matrix[i][j] = max(match, delete, insert)

            if score_matrix[i][j] == match:
                traceback_matrix[i][j] = 'diag'
            elif score_matrix[i][j] == delete:
                traceback_matrix[i][j] = 'up'
            else:
                traceback_matrix[i][j] = 'left'

    # Śledzenie wsteczne
    aligned_seq1 = ""
    aligned_seq2 = ""
    i, j = m, n
    while i > 0 or j > 0:
        if traceback_matrix[i][j] == 'diag':
            aligned_seq1 = seq1[i-1] + aligned_seq1
            aligned_seq2 = seq2[j-1] + aligned_seq2
            i -= 1
            j -= 1
        elif traceback_matrix[i][j] == 'up':
            aligned_seq1 = seq1[i-1] + aligned_seq1
            aligned_seq2 = "-" + aligned_seq2
            i -= 1
        else:  # 'left'
            aligned_seq1 = "-" + aligned_seq1
            aligned_seq2 = seq2[j-1] + aligned_seq2
            j -= 1

    return aligned_seq1, aligned_seq2


In [18]:
def read_fasta(file_path):
    sequences = [str(record.seq) for record in SeqIO.parse(file_path, "fasta")]
    return sequences


In [19]:
def main(fasta_file):
    sequences = read_fasta(fasta_file)

    if len(sequences) != 2:
        return "Plik FASTA powinien zawierać dokładnie dwie sekwencje."

    seq1, seq2 = sequences
    aligned_seq1, aligned_seq2 = needleman_wunsch(seq1, seq2, match_score=1, mismatch_penalty=-1, gap_penalty=-1)

    # Zapisz wyniki do pliku
    with open('alignment_result.txt', 'w') as output_file:
        output_file.write(f"{aligned_seq1}\n")
        output_file.write(f"{aligned_seq2}\n")

    return "Wynik dopasowania został zapisany do alignment_result.txt"
   # print(main(fasta_file))



In [20]:
from google.colab import files

uploaded = files.upload()
fasta_file = next(iter(uploaded))


Saving dwiesekwencje.fasta to dwiesekwencje (1).fasta


In [21]:
print(main(fasta_file))


Wynik dopasowania został zapisany do alignment_result.txt


In [22]:
files.download('alignment_result.txt')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>